In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt



def feature_importance_selection(indep_X, dep_Y, n):
    model = RandomForestClassifier(random_state=0)
    model.fit(indep_X, dep_Y)
    importances = model.feature_importances_
    indices = np.argsort(importances)[-n:]  # Top 'n' features
    selected_features = indep_X.iloc[:, indices]
    return selected_features, importances
    
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
 
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
   
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
    
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def featureImportance_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf): 
    
    dataframe=pd.DataFrame(index=['FeatureImportance'],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
        dataframe['SVMl'][idex]=accsvml[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['Navie'][idex]=accnav[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe

In [2]:
dataset1=pd.read_csv("churn_preprocess_data.csv",index_col=None)

df2=dataset1

df2 = pd.get_dummies(df2, drop_first=True)
df2.replace({True: 1, False: 0}, inplace=True)

indep_X=df2.drop('Churn_Yes', axis=1)
dep_Y=df2['Churn_Yes']

In [3]:
df2

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Bank transfer_Yes,Credit card_Yes,Electronic check_Yes,Mailed check_Yes,Month-to-month_Yes,One year_Yes,...,MultipleLines_Yes,InternetService_Yes,OnlineSecurity_Yes,OnlineBackup_Yes,DeviceProtection_Yes,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,PaperlessBilling_Yes,Churn_Yes
0,0,5,27.43,137.15,0,1,0,0,1,0,...,1,0,0,1,0,0,0,0,1,1
1,0,42,38.28,1607.76,0,0,1,0,0,0,...,0,1,1,1,0,0,0,0,0,1
2,0,61,106.44,6492.84,1,0,0,0,0,1,...,0,0,0,1,0,1,1,0,0,1
3,1,22,92.49,2034.78,0,0,1,0,1,0,...,0,0,0,0,1,0,0,1,0,0
4,1,21,19.63,412.23,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,61,42.52,2593.72,0,0,1,0,0,1,...,1,1,0,0,1,0,0,0,1,0
296,0,49,22.31,1093.19,0,0,0,1,1,0,...,0,1,0,0,1,0,1,1,1,1
297,0,71,59.57,4229.47,0,0,0,1,0,0,...,1,1,0,0,1,1,0,1,0,1
298,1,1,31.32,31.32,0,1,0,0,0,0,...,1,0,1,0,0,0,1,1,0,0


In [8]:
selected_features, feature_importances = feature_importance_selection(indep_X, dep_Y, 6)        

acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

In [9]:
feature_importances

array([0.03184152, 0.12616508, 0.14481426, 0.14912341, 0.02622129,
       0.02238907, 0.0198851 , 0.02624986, 0.02449533, 0.01967975,
       0.02414588, 0.03357526, 0.03114692, 0.03001939, 0.03017246,
       0.03146269, 0.02976966, 0.02976387, 0.03225057, 0.0286715 ,
       0.02666664, 0.02202405, 0.03123182, 0.02823462])

In [10]:
X_train, X_test, y_train, y_test = split_scalar(selected_features, dep_Y)    
    
        
classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
acclog.append(Accuracy)

classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
accsvml.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
accrf.append(Accuracy)
    
result=featureImportance_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)

In [7]:
result
#7

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
FeatureImportance,0.506667,0.52,0.493333,0.466667,0.533333,0.453333,0.413333


In [11]:
result
#6

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
FeatureImportance,0.52,0.506667,0.506667,0.506667,0.546667,0.413333,0.52


In [ ]:
result
#5

In [ ]:
result
#4

In [ ]:
result
#3

In [ ]:
result
#2